# Kunisch Features from ResNet architectures 
## Seminario de Tesis I, Primavera 2022 
### MDS Program. University of Chile.
#### Supervisor: Prof. Benjamín Bustos, Prof. Iván Sipirán
#### Author: Iván Sipirán, modified by Matías Vergara


## Imports

In [273]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Mounting Google Drive

In [274]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_path = 'drive/MyDrive/TesisMV/'
except:
    folder_path = '../'

In [275]:
os.listdir(folder_path)

['.git',
 '.gitignore',
 '.idea',
 'actas',
 'features',
 'graficos',
 'labels',
 'models',
 'notebooks',
 'patterns',
 'presentaciones',
 'README.md',
 'requirements.txt',
 'tablas']

## Dataset and model selection

In [276]:
#modify only this cell
USE_RN50 = True
SUBCHAPTERS = False
DS_FLAGS = ['blur']
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 1
RANDOM_TIMES = 2
ELASTIC_TIMES = 1

In [277]:
# This cells builds the data_flags variable, that will be used
# to map the requested data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

subchapter_str = 'subchapters/' if SUBCHAPTERS else ''
patterns_path = folder_path + 'patterns/' + subchapter_str + data_flags + "/"
labels_path = folder_path + 'labels/' +  subchapter_str + data_flags + "/"
# models_path = folder_path + 'models/' + subchapter_str + (f'resnet50_{data_flags}.pth' if USE_RN50 else f'resnet18_{data_flags}.pth')
# features_path = folder_path + 'features/' + subchapter_str + (f'resnet50_{data_flags}/' if USE_RN50 else f'resnet18_{data_flags}/')

rn = 50
ep = 75
models_path = folder_path + f"models/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}.pth"
features_path = folder_path + f"features/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}/"

if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation")

if not (os.path.isfile(models_path)):
    raise FileNotFoundError(f"No se encontró modelo para el conjunto de flags seleccionado. Verifique que el modelo exista y, de lo contrario, llame a ResNet Retraining\n{models_path}")


print("Pattern set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))
print("Modelo encontrado en {}".format(models_path))

Pattern set encontrado en ../patterns/blur/
Labels set encontrado en ../labels/blur/
Modelo encontrado en ../models/resnet50_blur_each5/resnet50_blur_e75.pth


## Dataset loader

In [278]:
class PatternDataset(Dataset):
    def __init__(self, root_dir, transform=None, build_classification=False, name_cla='output.cla'):
        self.root_dir = root_dir
        self.transform = transform
        self.namefiles = []


        self.classes = sorted(os.listdir(self.root_dir))

        for cl in self.classes:
            for pat in os.listdir(os.path.join(self.root_dir, cl)):
                self.namefiles.append((pat, cl))

        print(f'Files:{len(self.namefiles)}')
        self.namefiles = sorted(self.namefiles, key = lambda x: x[0])

        if build_classification:
            dictClasses = dict()

            for cl in self.classes:
                dictClasses[cl] = []

            for index, (name, cl) in enumerate(self.namefiles):
                dictClasses[cl].append((name, index))

            with open(name_cla, 'w') as f:
                f.write('PSB 1\n')
                f.write(f'{len(self.classes)} {len(self.namefiles)}\n')
                f.write('\n')
                for cl in self.classes:
                    f.write(f'{cl} 0 {len(dictClasses[cl])}\n')
                    for item in dictClasses[cl]:
                        f.write(f'{item[1]}\n')
                    f.write('\n')

    def __len__(self):
        return len(self.namefiles)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_name = os.path.join(self.root_dir, self.namefiles[index][1], self.namefiles[index][0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return self.namefiles[index], image


## Preparation

In [279]:
random.seed(30)

output_train = features_path + "augmented_train_df.json"
output_val = features_path + "val_df.json"
output_test = features_path + "test_df.json"

train_df = pd.read_json(labels_path + "augmented_train_df.json", orient='index')
val_df = pd.read_json(labels_path + "val_df.json", orient='index')
test_df = pd.read_json(labels_path + "test_df.json", orient='index')

train_pts = train_df.index.values
val_pts = val_df.index.values
test_pts = test_df.index.values

## Extraction

In [280]:
def imshow(inp, title = None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    plt.show()

def get_vector(model,layer, dim_embedding, x):

  my_embedding = torch.zeros(dim_embedding)

  def copy_data(m,i,o):
    my_embedding.copy_(o.data.squeeze())

  h = layer.register_forward_hook(copy_data)
  model(x)
  h.remove()

  return my_embedding

device = ('cuda' if torch.cuda.is_available() else 'cpu')

my_transform = transforms.Compose([ transforms.Resize(224),
                                    #transforms.CenterCrop(224),
                     transforms.ToTensor(),
                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
                    ])

dataTrain = PatternDataset(root_dir=os.path.join(patterns_path, 'train'), transform=my_transform)
dataVal = PatternDataset(root_dir=os.path.join(patterns_path, 'val'), transform=my_transform)
dataTest = PatternDataset(root_dir=os.path.join(patterns_path, 'test'), transform=my_transform)

loaderTrain = DataLoader(dataTrain)
loaderVal = DataLoader(dataVal)
loaderTest = DataLoader(dataTest)

if USE_RN50:
    model = models.resnet50(pretrained = True)
else:
    model = models.resnet18(pretrained = True)
dim = model.fc.in_features

output_dim = 20 if SUBCHAPTERS else 6
model.fc = nn.Linear(dim, output_dim)

model = model.to(device)

try:
    model.load_state_dict(torch.load(models_path))
except RuntimeError as e:
    print('Ignoring "' + str(e) + '"')

layer = model._modules.get('avgpool')

model.eval()

#features = []
features_train = {}
features_val = {}
features_test = {}


for name, img in loaderTrain:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_train[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderVal:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_val[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderTest:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_test[code] = feat.numpy().tolist()
#features = np.vstack(features)
#print(features.shape)

Files:1084
Files:78
Files:156


In [281]:
os.makedirs(features_path, exist_ok=True)

In [282]:
features_train_df = pd.DataFrame.from_dict(features_train, orient='index')
features_val_df = pd.DataFrame.from_dict(features_val, orient='index')
features_test_df = pd.DataFrame.from_dict(features_test, orient='index')

features_train_df.to_json(output_train, orient='index')
features_val_df.to_json(output_val, orient='index')
features_test_df.to_json(output_test, orient='index')

In [283]:
features_train_df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
10d,0.052288,0.151053,0.042946,0.072730,0.149786,0.212103,0.000000,0.007595,0.062738,0.017727,...,0.007520,0.019774,0.000000,0.008785,0.167395,0.003172,0.137184,0.005004,0.165087,0.017931
10d_blur,0.051012,0.148731,0.042817,0.073660,0.156056,0.209858,0.000000,0.006691,0.064340,0.018533,...,0.005919,0.026765,0.000000,0.009024,0.166809,0.003190,0.147681,0.004974,0.171514,0.015035
10e,0.177587,0.028253,0.096148,0.025770,0.182581,0.337547,0.000000,0.001634,0.000000,0.009844,...,0.008862,0.000382,0.008446,0.002322,0.267125,0.001751,0.145436,0.000972,0.072909,0.013215
10e_blur,0.181913,0.029316,0.099687,0.026923,0.188080,0.333517,0.000000,0.002352,0.000238,0.012671,...,0.009887,0.000257,0.005548,0.003593,0.267510,0.002174,0.147011,0.001075,0.078818,0.012300
11c,0.163801,0.060885,0.147545,0.295036,0.258510,0.097969,0.000000,0.007709,0.041882,0.087548,...,0.003977,0.056750,0.003197,0.004864,0.043767,0.000410,0.613212,0.009310,0.484576,0.007287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9b_blur,0.024879,0.084558,0.147060,0.128662,0.110811,0.220012,0.021198,0.276820,0.112341,0.009210,...,0.180046,0.066346,0.027325,0.078465,0.377176,0.008497,0.026694,0.068024,0.428789,0.214458
9c,0.012214,0.027661,0.165062,0.141493,0.124818,0.232926,0.021015,0.380179,0.115962,0.014127,...,0.208741,0.092676,0.019707,0.084391,0.515644,0.006331,0.259575,0.075970,0.582875,0.242759
9c_blur,0.009567,0.032335,0.236987,0.178250,0.194468,0.318602,0.020224,0.460661,0.167369,0.012675,...,0.276628,0.122632,0.028023,0.126640,0.449128,0.005528,0.230084,0.116580,0.553446,0.285442
9e,0.194008,0.084218,0.400226,0.229948,0.186748,0.440837,0.002013,0.493354,0.225457,0.010223,...,0.321207,0.047737,0.000000,0.191275,0.363328,0.004570,0.502535,0.152498,0.553976,0.284594
